In [1]:
# Load necessary python packages.
#import sys
#!{sys.executable} -m pip install matplotlib pandas cufflinks > /dev/null; # Remove > /dev/null in case of errors.


from ipywidgets import interact
import numpy as np
from numpy import log as ln
import matplotlib.pyplot as plt
import pandas as pd
import cufflinks as cf
import os

In [2]:
# Set the value of physical constants.           
Lv     = 2.45e6                # Latent heat of vaporisation [J/kg ]
gamma  = 0.066                 # Psychrometer constant [kPa/K]

# Set the parameters                   
dt     = 1.                    # Time step [h]
A      = 5000.                 # Surface area [m2]
d_max  = 0.                    # Maximum level (height above the surface) [m]
d_min  = -3.                   # Minimum level (depth below the surface)  [m]
V_max  = A * (d_max-d_min)     # Max volume [m3]

In [3]:
# Initial conditions
V_ini  = V_max           # The initial Volume [m3] 

# If the initial volume is bigger than the maximum volume of the pond, 
# set it to be equal to the maximum value.
if V_ini > V_max :
    V_ini = V_max

In [4]:
# Boundary conditions: Concentrations in inflowing water.
# You measured those in the Basic Skills Acquatic Ecology.
# Write them in order: concentration in precipitation C_N_P, concentration in lateral inflow C_N_l_in, 
# concentration in vertical inflow C_N_k_in. 
# All concentrations should be written in ug N/m3.
C_N_P     = 0.1e6   # ug/m3 Concentration of N in Precipitation.
C_N_Ql_in = 2.0e6   # ug/m3 Concentration of N in lateral inflow.
C_N_Qk_in = 0.      # ug/m3 Concentration of N in vertical inflow.    

In [5]:
# Load the meteorological data. 
# Load precipitation (P; mm), air temperature (Ta; degC) and global radiation (Rg; W m^-2).
teacher_dir        = os.getenv('TEACHER_DIR')
datapath           = teacher_dir + '/JHL_data/IC_data/'

df_meteo           = pd.read_csv(datapath+'Data_Meteo_Veenkampen_2013-2020.csv', sep=';', index_col='date-time', 
                                 usecols=['date-time', 'Rg', 'Ta', 'P'], parse_dates=['date-time'])

# Convert precipitation using the pond's surface area.
df_meteo['P'     ] = df_meteo['P']*A/1000.

In [6]:
# Calculate Makkink Evaporation
# Set up some constants and make a calculation for the saturation vapor pressure deficit es [kPa]
# the reference evaporation E_ref [kg m^2/s] and the crop factor for water Cx. 
es_0                                  = 0.6107      # kPa
a                                     = 7.5         #
b                                     = 237.3       # degC
Cx                                    = 1.25        # 'Crop' factor for water.
df_meteo['es'   ]                     = es_0*10**((a*df_meteo['Ta']) /(b+df_meteo['Ta']))
df_meteo['s'    ]                     = df_meteo['es']*ln(10)*a*b    /(b+df_meteo['Ta'])**2
df_meteo['E_ref']                     = 0.65*(1/Lv)*(df_meteo['s']/(df_meteo['s']+gamma))*df_meteo['Rg']

# Calculate the evaporation rate based on the Makkink approach (E_mak; m^3 h^-1).
# The units change between the reference evaporation and the actual evaporation rate.
df_meteo['E_mak']                     = Cx*df_meteo['E_ref']*3600.*A/1000.

In [7]:
# Below you can select whether or not you want to use your own output datafiles.
USE_MY_AMMONIA_DEPOSITION_FILE = True  # Advise: set to False, then it will use a file with F_NH3 over a water surface
USE_MY_WATER_BALANCE_FILE      = True   # Advise: set to True, to use the settings you made for Qlin and Qkin during IS 1.

# Select input files:
if USE_MY_AMMONIA_DEPOSITION_FILE:
    datapath      = '../BS_AirQuality/'
else:    
    datapath      = os.getenv('TEACHER_DIR') + '/JHL_data/IC_data/'
outfilename_BS_AQ = datapath + 'Data_Output_BasicSkills_AirQuality.csv'   # Use your own F_NH3 files, provided by us
print('You will use the following file:')
print(outfilename_BS_AQ)


# Load the dry deposition rate calculated in the Basic Skills Air Quality. 
# Load the dry deposition rate for the three stations i.e., Wekerom, Vredepeel and Zegveld
# as F_NH3_W, F_NH3_V, and F_NH3_Z, respectively. (ug/m2/s)
df_drydepo = pd.read_csv(outfilename_BS_AQ, sep=';', 
    index_col='date-time', usecols=['date-time', 'F_NH3_Wekerom', 'F_NH3_Vredepeel', 'F_NH3_Zegveld'], parse_dates=['date-time'])

# Calculate the wet deposition rate. Note that the precipitation is already converted
# from mm h$^{-1}$ to m$^3$ h$^{-1}$ using the pond's surface area. 
# We did this in Integration Skills - part 1.
df_meteo['dNdt_FNH3_wet'] = df_meteo['P'] * C_N_P

# Convert the units of the deposition rate to g/h. Therefore, set the area of the lake 
# (A; m^2), convert the seconds to hours, use the ratio 17/14 to convert the dry deposition 
# of NH3 to dry deposition of nitrogen N. 

df_drydepo['dNdt_FNH3_dry']   = df_drydepo['F_NH3_Wekerom'  ]*A*3600.*17./14.     # Use Wekerom
#df_drydepo['dNdt_FNH3_dry']  = df_drydepo['F_NH3_Vredepeel']*A*3600.*17./14.     # Use Vredepeel
#df_drydepo['dNdt_FNH3_dry']  = df_drydepo['F_NH3_Zegveld'  ]*A*3600.*17./14.     # Use Zegveld

You will use the following file:
../BS_AirQuality/Data_Output_BasicSkills_AirQuality.csv


In [8]:
# Add denitrification as a constant term (dNdt_den; ugN/h).
df_meteo['dNdt_den'] = 11.*14.*A*0.000001

In [9]:
# Calculate realistic values for Ql_in, Qk_in and Qk_out
# For horizontal conductivity: Go to Dinoloket.nl --> Ondergrondmodellen --> Select 'BRO Regis II v2.2. --> 
#   Zoom in to campus --> Select transect tool --> draw a transect. Finish the transect by double clicking
#   on the end point. --> Select 'Toon doorsnede' --> Mouseover the top layer and select 
#   'Horizontale doorlatendheid (kh)' --> Read the horizontal conductivity from the color scale.
# For vertical conductivity: In Dinoloket.nl (same transect): Mouseover the first impermeable layer below your pond -->
#   select 'Verticale doorlatendheid (kv)' --> Read out the vertical conductivity from the color scale.
#   select 'Dikte' --> Read out the thickness of the impermeable layer.
# For isohypses: Go to grondwatertools.nl --> Theme Grondwater --> Grondwaterstanden in beeld --> Isohypsen -->
#   Select WVP1 (Water conducting layer 1) --> Select a (not too recent) date & Select a (not too small) 
#   area including several observation points --> Read h2 (upstream), h1 (downstream), z1 (at the pond) 
#   and deltax from the map; Inspect seasonal variability!
#   Select WVP2 (Water conducting layer 2) --> Select area again --> Read z2 from the map

# ENTER REALISTIC VALUES FOR THE FORUM POND OR YOUR GROUP'S LAND UNIT HERE.
kh                             = 5.0/24.    # m/d --> m/h Horizontal conductivity
kv                             = 3.5e-3/24. # m/d --> m/h Vertical conductivity
Ah                             = 100.0      # m2 Vertical area of the area perpendicular to the horizontal water flow.
h2                             = 10.0       # m  Isohypse first  water conducting layer upstream
h1                             =  6.0       # m  Isohypse first  water conducting layer downstream
z1                             =  7.0       # m  Isohypse first  water conducting layer at the pond
z2                             =  8.0       # m  Isohypse second water conducting layer at the pond
deltax                         = 1200.      # m  Horizontal distance between h1 and h2.
deltaz                         = 10.0       # m  Thickness of impermeable layer (from Dinoloket)

df_meteo['Ql_in']              = Ah * kh/24. * (h2 - h1)/deltax   # m3/h. kh/24h to convert m/d to m/h. Add ditch water and/or overland flow.
df_meteo['Qk_in']              = A  * kv/24. * (z2 - z1)/deltaz   # m3/h Vertical inflow  (set to zero in case of infiltration)
df_meteo['Qk_out']             = 0.0                              # m3/h Vertical inflow (set to positive in case of infiltration)

In [10]:
# Run 1: Control run

# Initialise arrays.
df_meteo['V'     ]                      = V_ini    # m    initial volume
df_meteo['dV_dt' ]                      = 0.0      # m3/h initial water volume change in time 
df_meteo['Ql_out']                      = 0.       # m3/h initial lateral outflow

# Initial conditions
C_N_pond_ini                            = 2.5e6    #  Set initial N concentration in the pond (C_N_pond_ini; ug/m^3).
mN_ini                                  = C_N_pond_ini * V_ini

# Calculate lateral and vertical  inflows of N.
df_meteo['dNdt_Ql_in']                  = df_meteo['Ql_in'] * C_N_Ql_in # ug/h lateral  inflow
df_meteo['dNdt_Qk_in']                  = df_meteo['Qk_in'] * C_N_Qk_in # ug/h vertical inflow

# Initialise arrays containing variables. Set the tendency terms to 0 ugN/h at the first timestep.
it                                      = df_meteo.index[0]
df_meteo.at[it, 'mN'         ]          = mN_ini   # ug/m3  Initial N mass in the pond
df_meteo.at[it, 'dNdt_Ql_out']          = 0.       # ug N/h Tendency term due to lateral  outflow 
df_meteo.at[it, 'dNdt_Qk_out']          = 0.       # ug N/h Tendency term due to vertical outflow
df_meteo.at[it, 'dNdt_tot'   ]          = 0.       # ug N/h Tendency term (total)


# Time integration (solve the water and N mass balance equation)
nt                                      = len(df_meteo)
for it in range(1,nt):
    it_now                              = df_meteo.index[it  ]
    it_prev                             = df_meteo.index[it-1]
    
    # Calculate lateral outflow
    df_meteo.at[it_now, 'Ql_out']       = max(0., df_meteo.at[it_prev, 'V'] - V_max)

    # Calculate current N concentration in the pond and the resulting lateral and vertical outflow
    df_meteo.at[it_now, 'C_N_pond'   ]  = df_meteo.at[  it_prev,'mN']/df_meteo.at[it_prev,'V']
    df_meteo.at[it_now, 'dNdt_Ql_out']  = df_meteo.at[  it_now, 'Ql_out'] * df_meteo.at[it_now, 'C_N_pond'   ]
    df_meteo.at[it_now, 'dNdt_Qk_out']  = df_meteo.at[  it_now, 'Qk_out'] * df_meteo.at[it_now, 'C_N_pond'   ]

    # Calculate dV/dt from individual/disciplinary terms
    df_meteo.at[it_now, 'dV_dt']        = df_meteo.at[  it_now, 'P'            ] - df_meteo.at[it_now, 'E_mak'        ] \
                                        + df_meteo.at[  it_now, 'Qk_in'        ] - df_meteo.at[it_now, 'Qk_out'       ] \
                                        + df_meteo.at[  it_now, 'Ql_in'        ] - df_meteo.at[it_now, 'Ql_out'       ]
    
    # Calculate dmN/dt from individual/disciplinary terms
    df_meteo.at[it_now, 'dNdt_tot']     = df_drydepo.at[it_now, 'dNdt_FNH3_dry'] + df_meteo.at[it_now, 'dNdt_FNH3_wet'] \
                                        + df_meteo.at[  it_now, 'dNdt_Ql_in'   ] - df_meteo.at[it_now, 'dNdt_Ql_out'  ] \
                                        + df_meteo.at[  it_now, 'dNdt_Qk_in'   ] - df_meteo.at[it_now, 'dNdt_Qk_out'  ] \
                                        - df_meteo.at[  it_now, 'dNdt_den'     ]
    
    # Apply change (dV/dt) to total volume (V)
    df_meteo.at[it_now, 'V']            = df_meteo.at[it_prev, 'V']    + df_meteo.at[it_now, 'dV_dt']*dt 
    
    # Apply change (dmN/dt) to total N mass (mN)
    df_meteo.at[it_now, 'mN']           = df_meteo.at[ it_prev,'mN']   + df_meteo.at[it_now, 'dNdt_tot']*dt     

In [12]:
# Run 2: Sensitivity run

# Make a copy of df_meteo and call it df_sens1 (or df_sens2)
df_sens1                                = df_meteo.copy()

# You can now change any value in df_sens1, for example, if you want to double the precipitation:
df_sens1['P']                           = df_meteo['P'] * 2.0

# Initialise arrays.
df_sens1['V'     ]                      = V_ini    # m    initial volume
df_sens1['dV_dt' ]                      = 0.0      # m3/h initial water volume change in time 
df_sens1['Ql_out']                      = 0.       # m3/h initial lateral outflow

# Initial conditions
C_N_pond_ini                            = 2.5e6    #  Set initial N concentration in the pond (C_N_pond_ini; ug/m^3).
mN_ini                                  = C_N_pond_ini * V_ini

# Calculate lateral and vertical  inflows of N.
df_sens1['dNdt_Ql_in']                  = df_sens1['Ql_in'] * C_N_Ql_in # ug/h lateral  inflow
df_sens1['dNdt_Qk_in']                  = df_sens1['Qk_in'] * C_N_Qk_in # ug/h vertical inflow

# Initialise arrays containing variables. Set the tendency terms to 0 ugN/h at the first timestep.
it                                      = df_sens1.index[0]
df_sens1.at[it, 'mN'         ]          = mN_ini   # ug/m3  Initial N mass in the pond
df_sens1.at[it, 'dNdt_Ql_out']          = 0.       # ug N/h Tendency term due to lateral  outflow 
df_sens1.at[it, 'dNdt_Qk_out']          = 0.       # ug N/h Tendency term due to vertical outflow
df_sens1.at[it, 'dNdt_tot'   ]          = 0.       # ug N/h Tendency term (total)


# Time integration (solve the water and N mass balance equation)
nt                                      = len(df_sens1)
for it in range(1,nt):
    it_now                              = df_sens1.index[it  ]
    it_prev                             = df_sens1.index[it-1]
    
    # Calculate lateral outflow
    df_sens1.at[it_now, 'Ql_out']       = max(0., df_sens1.at[it_prev, 'V'] - V_max)

    # Calculate current N concentration in the pond and the resulting lateral and vertical outflow
    df_sens1.at[it_now, 'C_N_pond'   ]  = df_sens1.at[  it_prev,'mN']/df_meteo.at[it_prev,'V']

    df_sens1.at[it_now, 'dNdt_Ql_out']  = df_sens1.at[  it_now, 'Ql_out'] * df_sens1.at[it_now, 'C_N_pond'   ]
    df_sens1.at[it_now, 'dNdt_Qk_out']  = df_sens1.at[  it_now, 'Qk_out'] * df_sens1.at[it_now, 'C_N_pond'   ]

    # Calculate dV/dt from individual/disciplinary terms
    df_sens1.at[it_now, 'dV_dt']        = df_sens1.at[  it_now, 'P'            ] - df_sens1.at[it_now, 'E_mak'        ] \
                                        + df_sens1.at[  it_now, 'Qk_in'        ] - df_sens1.at[it_now, 'Qk_out'       ] \
                                        + df_sens1.at[  it_now, 'Ql_in'        ] - df_sens1.at[it_now, 'Ql_out'       ]
    
    # Calculate dmN/dt from individual/disciplinary terms
    df_sens1.at[it_now, 'dNdt_tot']     = df_drydepo.at[it_now, 'dNdt_FNH3_dry'] + df_sens1.at[it_now, 'dNdt_FNH3_wet'] \
                                        + df_sens1.at[  it_now, 'dNdt_Ql_in'   ] - df_sens1.at[it_now, 'dNdt_Ql_out'  ] \
                                        + df_sens1.at[  it_now, 'dNdt_Qk_in'   ] - df_sens1.at[it_now, 'dNdt_Qk_out'  ] \
                                        - df_sens1.at[  it_now, 'dNdt_den'     ]
    
    # Apply change (dV/dt) to total volume (V)
    df_sens1.at[it_now, 'V']            = df_sens1.at[it_prev, 'V']    + df_sens1.at[it_now, 'dV_dt']*dt 
    
    # Apply change (dmN/dt) to total N mass (mN)
    df_sens1.at[it_now, 'mN']           = df_sens1.at[ it_prev,'mN']   + df_sens1.at[it_now, 'dNdt_tot']*dt   

In [11]:
# Calculate the Chlorophyl concentration and Secchi depth, based on the control run


# Set up the initial chlorofyl value.
df_chlorophyl     = df_meteo.copy()[[]] #copy df1 and erase all columns
Chlorophyl_ini    = 1e-4            # ug/l
df_chlorophyl['Chlorophyl'] = Chlorophyl_ini

# Parameter settings.
r                 = 0.5             # Growth rate
l                 = 0.005           # Loss rate
Tmin              = 0.0             # oC Min temperature for algae growth
Topt              = 20.0            # oC Optimal temperature
Tmax              = 40.0            # oC Max temperature for algae growth
hN                = 0.40            # Sensitivity of algae growth to N concentration
hP                = 0.15            # Sensitivity of algae growth to P concentration
ha                = 15.             # Crowding factor
depth             = 1.5             # Depth in m

# Temperature limitation.
Ta                = df_meteo['Ta'] # =IF(H9>0;IF(H9<Topt;H9*1/Topt;1-(H9-Topt)*1/Topt);0)
Tlim              = Ta * 0.0        # init
Tlim[(Ta >  Tmin) & (Ta < Topt)] = Ta/Topt
Tlim[(Ta >= Topt) & (Ta < Tmax)] = 1-(Ta-Topt)/Topt

# Nitrogen limitation.
Llim              = df_meteo['Rg'].copy()
Llim[df_meteo['Rg'] >  800] = 1
Llim[df_meteo['Rg'] <= 800] = df_meteo['Rg']/800.
C_N               = df_meteo['C_N_pond']  #* from_ugperm3_to_mgperl
Nlim              = C_N  / (hN + C_N)

# Phosphorus limitation.
C_P               = 5   # ug/l (fixed value for now, adapt if you have better measurements)
Plim              = C_P / (hP + C_P)

# Nutrient limitation.
Nutlim            = np.minimum(Nlim,Plim)

# Time integration.
nt = len(df_chlorophyl)
for it in range(1,nt):
    it_now  = df_chlorophyl.index[it  ]
    it_prev = df_chlorophyl.index[it-1]
      
    df_chlorophyl.at[it_now, 'Chlorophyl'] = df_chlorophyl.at[it_prev, 'Chlorophyl'] \
        + (r * df_chlorophyl.at[it_prev,'Chlorophyl'] * Llim.at[it_now] * Tlim[it_now] * \
                                                         Nutlim[it_now] * (ha/(ha+df_chlorophyl.at[it_prev, 'Chlorophyl']))) \
                                                      -  l * df_chlorophyl.at[it_prev,'Chlorophyl']
    
df_chlorophyl['Secchi'] = np.minimum(10**(0.65-0.43*np.log10(df_chlorophyl['Chlorophyl'])), depth)

In [ ]:
# Plot the time series of precipitation. 
# Plot Time [year] on x-axis and Precipitation [m^3/h] on y-axis. 
fig1 = df_meteo['P'].groupby(pd.Grouper(freq='Q')).mean().iplot(asFigure=True, xTitle='Time [year]', yTitle='Precipitation [m3/h]', width=2) 
fig1.show()

In [ ]:
# Save the volumes in the same data frame for easier plotting.
df_volume = pd.concat([df_meteo['V'], df_sens1['V'], ], axis=1, sort=False)
df_volume.columns = ['V control','V sensitivity 1']

# Plot the time series of calculated water volume. 
# Plot Time [year] on x-axis and Volume [m^3] on y-axis. 
# Note that for easier recogniotion we divided the volume by 1000. 
fig2 = (df_volume/1000).iplot(asFigure=True, xTitle='Time [year]', yTitle='Volume [1e3 m3]', width=2) 
fig2.show()

In [ ]:
df_C_N         = pd.concat([df_meteo['C_N_pond'], df_sens1['C_N_pond']],axis=1, sort=False)
df_C_N.columns = ['Control', 'Sensitivity 1']

fig3 = (df_C_N).iplot(asFigure=True, xTitle='Time [year]', yTitle='Concentration [ug/m3]', width=2) 
fig3.show()

In [ ]:
fig4 = df_chlorophyl[['Chlorophyl', 'Secchi']].iplot(asFigure=True, subplots=True, shape=(2,1), shared_xaxes=True,
    layout=dict(yaxis=dict(title='Chlorophyl [ug/l]'), xaxis=dict(title=' ')))
fig4['layout']['yaxis2'].update({'title':'Secchi [m]'})
fig4['layout']['xaxis2'].update({'title':'Time [years]'})
fig4.show()